In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [ ]:
def alpha_m(v):
    return 0

In [ ]:
def HodgkinHuxley_2D(t,y):
    v = y[0]
    n = y[1]

    dvdt = ( g_Na * m_inf(v)**3 * (0.83 - n) * (v_Na - v) + g_K * n**4*(v_K - v) + g_L * (v_L - v) + I_app ) / c
    dndt = (n_inf(v) - n) / tau_n

    return np.array([dvdt, dndt])